# Getting Started

TODO's / Notes:

* Slides: https://docs.google.com/presentation/d/1I5ID4K7i97K1b5XOVgExIbLb_b-0UBVcR15zm3PHU2o/edit#slide=id.g6c52a2e8d8_0_177
    * [ ] Update (Nice to have), with PO.DAAC concept ID updates
    * [ ] Update with openscapes template
    * [ ] Add call to action at end of slides
    
* [ ] Move Kerchunk notebook to gallery repo 
    
    
* Three notebooks:
    * [ ] Intro
    * [ ] Advanced features:
        * [ ] Kerchunk: https://gist.github.com/betolink/4f3fc3c0b04286ece8434d85dab0b932
        * [ ] Coming soon: explore()
        


__

Outline notes: 

* Three lines of code to get the data

* More details on spatial search options, temporal search, keyword search

    * Emphasize reproducibility 
    * earthaccess as a complement to other search tools like Earthdata Search

* Show both download to local machine and s3 access: Same code

* Coming features / roadmap
    * earthaccess.explore()
        * Even though .explore() won’t be fully merged, it’s still good to show upcoming features
    * Kerchunk

* Call to action
    * Make it clear that this is a community effort; highlight what’s already been done